In [3]:
# Load Library

packages <- c("dplyr", "haven","ggplot2","MASS", "randomForest", "caret", "ranger", "parallel")

install_if_missing <- function(pkg) {
  if (!requireNamespace(pkg, quietly = TRUE)) {
    install.packages(pkg)
  }
  library(pkg, character.only = TRUE)
}

lapply(packages, install_if_missing)

select <- dplyr::select

[[1]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[2]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[3]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[4]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[5]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[6]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[7]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"        

[[8]]
 [1] "parallel"     "ranger"       "caret"        "lattice"      "randomForest"
 [6] "MASS"         "ggplot2"      "haven"        "dplyr"        "stats"       
[11] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[16] "base"

# Maching Learning

In [4]:
df1 <- read.csv("Data/merge_df_cln.csv")
df1 <- df1 %>% 
        mutate(hyper = ifelse((bp_sys_mean >= 140 | bp_dia_mean >= 90), 1 , 0)) %>% 
        filter(htn_aware == "No") %>% #filter with Aware
        dplyr::select(-"X", -"bp_sys_mean", -"svy_id", -"bp_dia_mean", -"htn_aware")

colnames(df1)
# df1$hyper <- as.factor(df1$hyper)
# summary(df1$hyper)

[1] "svy_year"        "demo_age_cat"    "demo_race"       "demo_race_black"
 [5] "demo_age_years"  "demo_pregnant"   "demo_gender"     "cc_bmi"         
 [9] "DRXTPROT"        "DRXTCARB"        "DRXTTFAT"        "DRXTSFAT"       
[13] "DRXTMFAT"        "DRXTPFAT"        "DRXTCHOL"        "DRXTFIBE"       
[17] "DRXTVB6"         "DRXTVB12"        "DRXTFOLA"        "DRXTVC"         
[21] "DRXTCALC"        "DRXTPHOS"        "DRXTMAGN"        "DRXTIRON"       
[25] "DRXTZINC"        "DRXTCOPP"        "DRDTSODI"        "DRXTPOTA"       
[29] "DRXTSELE"        "DRXTCAFF"        "DRXTTHEO"        "DRXTALCO"       
[33] "DRXTMOIS"        "DRXTS040"        "DRXTS060"        "DRXTS080"       
[37] "DRXTS100"        "DRXTS120"        "DRXTS140"        "DRXTS160"       
[41] "DRXTS180"        "DRXTM161"        "DRXTM181"        "DRXTM201"       
[45] "DRXTM221"        "DRXTP182"        "DRXTP183"        "DRXTP184"       
[49] "DRXTP204"        "DRXTP205"        "DRXTP225"        "DRXTP226"       
[53] "DRD320GW"        "hyper"

## Random Forest

### Desrease size of majority

In [5]:
table(df1$hyper)

# Separate majority and minority classes
df1_majority <- df1 %>% filter(hyper == 0)
df1_minority <- df1 %>% filter(hyper == 1)

set.seed(42)
# Oversample minority class (duplicate existing data)
df1_majority_oversampled <- df1_majority %>% sample_n(nrow(df1_minority), replace = F)

# Combine balanced dataset
df1_balanced_decrease <- bind_rows(df1_minority, df1_majority_oversampled)

# Check new class distribution
table(df1_balanced_decrease$hyper)


    0     1 
29845  3680 


   0    1 
3680 3680 

In [6]:
# Change here for different resample method
df <- df1_balanced_decrease


# Split into training (70%) and testing (30%) sets
set.seed(42)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

In [7]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data, 
                   num.trees = 500, 
                   mtry = sqrt(ncol(train_data) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 754 253
         1 350 851
                                          
               Accuracy : 0.7269          
                 95% CI : (0.7078, 0.7454)
    No Information Rate : 0.5             
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4538          
                                          
 Mcnemar's Test P-Value : 9.252e-05       
                                          
            Sensitivity : 0.6830          
            Specificity : 0.7708          
         Pos Pred Value : 0.7488          
         Neg Pred Value : 0.7086          
             Prevalence : 0.5000          
         Detection Rate : 0.3415          
   Detection Prevalence : 0.4561          
      Balanced Accuracy : 0.7269          
                                          
       'Positive' Class : 0               
         

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRXTCALC 
     351.108904      240.962979       49.696176       43.719252       39.300271 
         DRXTVC        DRDTSODI        DRXTCAFF         DRXTVB6        DRXTPOTA 
      39.223848       38.839815       38.618285       38.537601       38.515220 
       DRXTFOLA        DRXTM201        DRXTCHOL        DRXTVB12        DRXTPROT 
      38.267965       38.192021       37.378188       37.225468       36.854727 
       DRXTS120        DRXTP204        DRXTIRON        DRXTSELE        DRXTZINC 
      36.662841       36.598173       36.270201       36.147135       36.105218 
       DRXTFIBE        DRXTP183        DRXTCOPP        DRXTPHOS        DRXTM161 
      35.723475       35.480631       35.416426       35.333891       35.043836 
       DRXTP182        DRXTM181        DRD320GW        DRXTPFAT        DRXTS140 
      35.005980       34.804971       34.663185       34.545867       34.258294 
       DRXTMAGN        DRXTMFAT          cc_bmi        DRXTS180        DRXTTFAT 
      34.040644       33.825261       33.200892       32.975236       32.927762 
       DRXTS080        DRXTS160        DRXTS040        DRXTSFAT        DRXTS100 
      32.707109       32.230404       31.736682       31.581750       31.065319 
       DRXTS060        DRXTM221        DRXTP226        DRXTP225        DRXTP205 
      29.292366       28.672196       26.963092       25.802190       24.353936 
       DRXTTHEO        svy_year        DRXTALCO     demo_gender        DRXTP184 
      23.466477       22.300750       20.996893       17.276316       15.741951 
      demo_race demo_race_black   demo_pregnant 
      12.744057        8.226591        0.000000

### Increase size of minority (Bootstrap)

In [8]:
table(df1$hyper)

# Separate majority and minority classes
df1_majority <- df1 %>% filter(hyper == 0)
df1_minority <- df1 %>% filter(hyper == 1)

set.seed(24)
# Oversample minority class (duplicate existing data)
df1_minority_oversampled <- df1_minority %>% sample_n(nrow(df1_majority), replace = T)

# Combine balanced dataset
df1_balanced_bootstrap <- bind_rows(df1_majority, df1_minority_oversampled)

# Check new class distribution
table(df1_balanced_bootstrap$hyper)


    0     1 
29845  3680 


    0     1 
29845 29845 

In [9]:
# Change here for different resample method
df <- df1_balanced_bootstrap


# Split into training (70%) and testing (30%) sets
set.seed(42)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

In [10]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data, 
                   num.trees = 500, 
                   mtry = sqrt(ncol(train_data) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 8587   42
         1  366 8911
                                          
               Accuracy : 0.9772          
                 95% CI : (0.9749, 0.9794)
    No Information Rate : 0.5             
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.9544          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       
                                          
            Sensitivity : 0.9591          
            Specificity : 0.9953          
         Pos Pred Value : 0.9951          
         Neg Pred Value : 0.9605          
             Prevalence : 0.5000          
         Detection Rate : 0.4796          
   Detection Prevalence : 0.4819          
      Balanced Accuracy : 0.9772          
                                          
       'Positive' Class : 0               
   

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRDTSODI 
     2907.15194      2129.98151       461.88215       389.60978       376.15928 
       DRXTCAFF        DRXTM201        DRXTFOLA          DRXTVC         DRXTVB6 
      363.95513       361.80278       361.11083       360.24186       353.33623 
       DRXTIRON        DRXTCALC        DRXTSELE        DRXTVB12        DRXTCHOL 
      352.45775       348.91503       347.24242       347.09478       345.60386 
       DRXTPOTA        DRXTCOPP        DRXTFIBE        DRXTM161        DRXTP183 
      343.34628       341.53366       340.71433       339.73281       334.82962 
       DRD320GW        DRXTPROT        DRXTP204        DRXTS120        DRXTZINC 
      332.34617       331.03937       329.69825       328.25522       326.46229 
       DRXTPHOS        DRXTMAGN          cc_bmi        DRXTPFAT        DRXTP182 
      325.82961       324.20442       321.17728       317.17492       316.58409 
       DRXTM181        DRXTMFAT        DRXTS180        DRXTS140        DRXTTFAT 
      307.17943       304.68294       302.22000       294.30666       292.61452 
       DRXTS080        DRXTS040        DRXTS160        DRXTS100        DRXTSFAT 
      290.05910       286.49208       286.13853       279.88913       267.21040 
       DRXTS060        DRXTM221        DRXTP226        DRXTP225        DRXTALCO 
      266.37420       254.29507       253.13222       235.32351       234.32940 
       DRXTP205        DRXTTHEO        svy_year     demo_gender        DRXTP184 
      233.91426       221.19929       204.83640       153.71445       151.66398 
      demo_race demo_race_black   demo_pregnant 
      122.42245        95.64153         0.00000

### Hybrid

In [11]:
# Step 1: Reduce the majority class (Under-sampling)
df1_majority <- df1 %>% filter(hyper == 0) %>% sample_n(nrow(df1_minority) * 1.5, replace = FALSE)

# Step 2: Increase the minority class (Over-sampling)
df1_minority_oversampled <- df1_minority %>% sample_n(nrow(df1_majority), replace = TRUE)

# Combine both
df1_balanced_hybrid <- bind_rows(df1_majority, df1_minority_oversampled)

# Check new class distribution
table(df1_balanced_hybrid$hyper)



   0    1 
5520 5520 

In [12]:
# Change here for different resample method
df <- df1_balanced_hybrid


# Split into training (70%) and testing (30%) sets
set.seed(24)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

In [13]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data, 
                   num.trees = 1500, 
                   mtry = floor(sqrt(ncol(train_data) - 1) * 1.5), 
                   importance = "impurity",
                   probability = TRUE,
                   num.threads = parallel::detectCores())


# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# # Convert probabilities to class labels
# predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
# predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# # Model Evaluation
# conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# # Print Accuracy, Sensitivity, Specificity
# print(conf_matrix)

threshold <- 0.45  # Lower threshold slightly to increase sensitivity

predicted_class <- ifelse(predictions[,2] > threshold, "1", "0")  
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)
print(conf_matrix)


# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 1213  146
         1  443 1510
                                         
               Accuracy : 0.8222         
                 95% CI : (0.8087, 0.835)
    No Information Rate : 0.5            
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.6443         
                                         
 Mcnemar's Test P-Value : < 2.2e-16      
                                         
            Sensitivity : 0.7325         
            Specificity : 0.9118         
         Pos Pred Value : 0.8926         
         Neg Pred Value : 0.7732         
             Prevalence : 0.5000         
         Detection Rate : 0.3662         
   Detection Prevalence : 0.4103         
      Balanced Accuracy : 0.8222         
                                         
       'Positive' Class : 0              
                       

demo_age_years    demo_age_cat        DRXTMOIS        DRXTM201          cc_bmi 
      576.90182       338.12235        74.12085        68.43915        67.58800 
       DRXTCARB        DRXTCAFF          DRXTVC        DRXTFOLA        DRXTIRON 
       66.27091        64.35693        64.33623        63.82393        63.07931 
       DRDTSODI        DRXTCALC         DRXTVB6        DRD320GW        DRXTVB12 
       62.32662        61.24150        60.31909        58.89723        58.71496 
       DRXTCOPP        DRXTFIBE        DRXTSELE        DRXTM161        DRXTP183 
       58.56283        57.77465        57.34048        57.05965        56.97429 
       DRXTZINC        DRXTPOTA        DRXTMAGN        DRXTS120        DRXTP182 
       56.60522        56.45465        56.03827        54.17794        54.17672 
       DRXTCHOL        DRXTM181        DRXTP204        DRXTMFAT        DRXTPROT 
       53.95844        53.86605        53.20253        52.17103        52.12388 
       DRXTPFAT        DRXTS180        DRXTS140        DRXTPHOS        DRXTS040 
       51.45075        51.41236        49.90748        49.66990        48.38974 
       DRXTS080        DRXTM221        DRXTTFAT        DRXTS160        DRXTS100 
       48.22377        46.86947        46.41165        46.07705        45.78139 
       DRXTSFAT        DRXTS060        DRXTP226        DRXTALCO        DRXTP205 
       45.41762        44.48161        42.41925        40.50937        39.91412 
       DRXTP225        DRXTTHEO        svy_year     demo_gender        DRXTP184 
       39.06128        38.16700        33.49640        27.67994        25.75719 
      demo_race demo_race_black   demo_pregnant 
       23.08381        17.69551         0.00000

In [14]:
# Change here for different resample method
df <- df1


# Split into training (70%) and testing (30%) sets
set.seed(42)
train_index <- createDataPartition(df$hyper, p = 0.7, list = FALSE)
train_data <- df[train_index, ]
test_data <- df[-train_index, ]

train_data$hyper <- as.factor(train_data$hyper)
test_data$hyper <- as.factor(test_data$hyper)

### Desrease size of majority

In [15]:
table(train_data$hyper)

# Separate majority and minority classes
train_data_majority <- train_data %>% filter(hyper == 0)
train_data_minority <- train_data %>% filter(hyper == 1)

set.seed(42)
# Oversample minority class (duplicate existing data)
train_data_majority_oversampled <- train_data_majority %>% sample_n(nrow(train_data_minority), replace = F)

# Combine balanced dataset
train_data_decrease <- bind_rows(train_data_minority, train_data_majority_oversampled)

# Check new class distribution
table(train_data_decrease$hyper)


    0     1 
20859  2609 


   0    1 
2609 2609 

In [16]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data_decrease, 
                   num.trees = 1000, 
                   mtry = sqrt(ncol(train_data_decrease) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 6282  262
         1 2704  809
                                         
               Accuracy : 0.7051         
                 95% CI : (0.6961, 0.714)
    No Information Rate : 0.8935         
    P-Value [Acc > NIR] : 1              
                                         
                  Kappa : 0.2268         
                                         
 Mcnemar's Test P-Value : <2e-16         
                                         
            Sensitivity : 0.6991         
            Specificity : 0.7554         
         Pos Pred Value : 0.9600         
         Neg Pred Value : 0.2303         
             Prevalence : 0.8935         
         Detection Rate : 0.6246         
   Detection Prevalence : 0.6507         
      Balanced Accuracy : 0.7272         
                                         
       'Positive' Class : 0              
                       

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRDTSODI 
     361.372716      239.873059       48.820051       46.426725       41.860177 
       DRXTCALC          DRXTVC        DRXTZINC        DRXTFOLA        DRXTIRON 
      40.849093       39.467798       38.856823       38.718219       38.538928 
       DRXTCHOL        DRXTM201        DRXTCAFF         DRXTVB6        DRXTPROT 
      38.074662       38.054854       37.734723       37.731070       37.338423 
       DRXTFIBE        DRXTPOTA        DRD320GW        DRXTMAGN        DRXTVB12 
      37.315580       37.256212       37.124884       36.922223       36.760060 
       DRXTCOPP        DRXTSELE        DRXTP204        DRXTM161        DRXTP183 
      36.531094       36.201222       36.046718       35.910416       35.763108 
       DRXTPHOS        DRXTS120        DRXTPFAT        DRXTP182          cc_bmi 
      35.742774       35.087349       34.176336       34.093789       33.550310 
       DRXTS180        DRXTMFAT        DRXTS160        DRXTM181        DRXTS140 
      33.433856       33.104298       32.628041       32.361355       31.695634 
       DRXTTFAT        DRXTS040        DRXTS080        DRXTSFAT        DRXTS100 
      31.488687       31.434897       31.240482       31.216998       30.777044 
       DRXTP226        DRXTS060        DRXTM221        DRXTP225        DRXTP205 
      30.610603       29.781249       28.379791       28.160397       25.680672 
       svy_year        DRXTALCO        DRXTTHEO     demo_gender        DRXTP184 
      24.243674       23.211938       22.857376       17.597420       16.211094 
      demo_race demo_race_black   demo_pregnant 
      13.907005        7.180378        0.000000

### Increase size of minority (Bootstrap)

In [17]:
table(train_data$hyper)

# Separate majority and minority classes
train_data_majority <- train_data %>% filter(hyper == 0)
train_data_minority <- train_data %>% filter(hyper == 1)

set.seed(24)
# Oversample minority class (duplicate existing data)
train_data_minority_oversampled <- train_data_minority %>% sample_n(nrow(train_data_majority), replace = T)

# Combine balanced dataset
train_data_bootstrap <- bind_rows(train_data_majority, train_data_minority_oversampled)

# Check new class distribution
table(train_data_bootstrap$hyper)


    0     1 
20859  2609 


    0     1 
20859 20859 

In [18]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data_bootstrap, 
                   num.trees = 500, 
                   mtry = sqrt(ncol(train_data_bootstrap) - 1), 
                   importance = "impurity",
                   probability = TRUE,  # Enables probability prediction
                   num.threads = num_cores)

# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

# Convert probabilities to class labels
predicted_class <- ifelse(predictions[,2] > 0.5, "1", "0")  # Threshold at 0.5
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

# Model Evaluation
conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)

# Print Accuracy, Sensitivity, Specificity
print(conf_matrix)

# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 8860  961
         1  126  110
                                          
               Accuracy : 0.8919          
                 95% CI : (0.8857, 0.8979)
    No Information Rate : 0.8935          
    P-Value [Acc > NIR] : 0.7042          
                                          
                  Kappa : 0.1351          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.9860          
            Specificity : 0.1027          
         Pos Pred Value : 0.9021          
         Neg Pred Value : 0.4661          
             Prevalence : 0.8935          
         Detection Rate : 0.8810          
   Detection Prevalence : 0.9765          
      Balanced Accuracy : 0.5443          
                                          
       'Positive' Class : 0               
   

demo_age_years    demo_age_cat        DRXTMOIS        DRXTCARB        DRDTSODI 
     3090.98318      1946.85794       454.11640       430.73752       382.37954 
         DRXTVC        DRXTCAFF        DRXTM201        DRXTFOLA        DRD320GW 
      377.27856       374.64419       371.59840       367.43855       358.96206 
       DRXTCALC        DRXTIRON        DRXTM161        DRXTSELE        DRXTFIBE 
      358.44254       354.97379       354.44109       354.43501       351.94754 
       DRXTVB12         DRXTVB6        DRXTPOTA        DRXTCHOL        DRXTP183 
      350.14456       350.13290       348.75954       345.36647       343.39816 
       DRXTCOPP        DRXTZINC        DRXTPHOS        DRXTP204        DRXTPROT 
      342.75835       341.68432       339.94815       339.03432       337.42184 
       DRXTP182        DRXTS120        DRXTMAGN        DRXTPFAT        DRXTM181 
      331.35486       331.33988       324.38721       321.91466       317.31252 
       DRXTS180        DRXTMFAT          cc_bmi        DRXTS140        DRXTS040 
      314.56203       310.83621       307.18957       301.95701       294.49135 
       DRXTS100        DRXTS080        DRXTTFAT        DRXTS160        DRXTSFAT 
      294.24201       293.31942       290.19777       286.57055       276.15351 
       DRXTS060        DRXTM221        DRXTP226        DRXTP225        DRXTP205 
      275.96052       270.52464       256.51427       250.67242       237.49248 
       DRXTTHEO        svy_year        DRXTALCO     demo_gender        DRXTP184 
      222.93793       214.53537       184.01579       176.91708       156.55667 
      demo_race demo_race_black   demo_pregnant 
      125.52950        75.78033         0.00000

### Hybrid

In [19]:
# Step 1: Reduce the majority class (Under-sampling)
train_data_majority <- train_data %>% filter(hyper == 0) %>% sample_n(nrow(train_data_minority) * 1.5, replace = FALSE)

# Step 2: Increase the minority class (Over-sampling)
train_data_minority_oversampled <- train_data_minority %>% sample_n(nrow(train_data_majority), replace = TRUE)

# Combine both
train_data_hybrid <- bind_rows(train_data_majority, train_data_minority_oversampled)

# Check new class distribution
table(train_data_hybrid$hyper)



   0    1 
3913 3913 

In [20]:
# Check available CPU cores
num_cores <- detectCores()
print(paste("Using", num_cores, "cores"))

# Train Random Forest classification model with multi-threading
rf_model <- ranger(hyper ~ ., data = train_data_hybrid, 
                   num.trees = 1500, 
                   mtry = floor(sqrt(ncol(train_data_hybrid) - 1) * 1.5), 
                   importance = "impurity",
                   probability = TRUE,
                   num.threads = parallel::detectCores())


# Predict class probabilities on the test set
predictions <- predict(rf_model, test_data)$predictions

threshold <- 0.45  # Lower threshold slightly to increase sensitivity

predicted_class <- ifelse(predictions[,2] > threshold, "1", "0")  
predicted_class <- factor(predicted_class, levels = levels(test_data$hyper))

conf_matrix <- confusionMatrix(predicted_class, test_data$hyper)
print(conf_matrix)


# Variable Importance Plot
importance_values <- rf_model$variable.importance
sort(importance_values, decreasing = T)

[1] "Using 32 cores"
Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 6597  313
         1 2389  758
                                        
               Accuracy : 0.7313        
                 95% CI : (0.7226, 0.74)
    No Information Rate : 0.8935        
    P-Value [Acc > NIR] : 1             
                                        
                  Kappa : 0.2384        
                                        
 Mcnemar's Test P-Value : <2e-16        
                                        
            Sensitivity : 0.7341        
            Specificity : 0.7077        
         Pos Pred Value : 0.9547        
         Neg Pred Value : 0.2409        
             Prevalence : 0.8935        
         Detection Rate : 0.6560        
   Detection Prevalence : 0.6871        
      Balanced Accuracy : 0.7209        
                                        
       'Positive' Class : 0             
                                        


demo_age_years    demo_age_cat        DRXTMOIS          cc_bmi        DRXTCARB 
      558.08330       407.23980        85.02789        75.67742        71.23447 
       DRDTSODI          DRXTVC        DRXTCAFF        DRD320GW        DRXTCALC 
       68.62934        66.26111        65.92920        64.27032        64.17680 
       DRXTFOLA        DRXTM201        DRXTM161        DRXTSELE        DRXTFIBE 
       63.96652        61.66891        61.04947        60.94667        60.28827 
       DRXTPOTA        DRXTZINC         DRXTVB6        DRXTMAGN        DRXTIRON 
       60.00165        59.49312        59.44929        59.18044        58.94923 
       DRXTVB12        DRXTCOPP        DRXTP204        DRXTCHOL        DRXTPHOS 
       58.76315        57.64285        56.94394        55.93081        55.18266 
       DRXTP183        DRXTPROT        DRXTS120        DRXTP182        DRXTPFAT 
       54.73481        54.34232        53.86023        51.24984        50.99072 
       DRXTMFAT        DRXTM181        DRXTS140        DRXTS180        DRXTM221 
       50.92965        50.76325        48.54064        48.44617        47.54714 
       DRXTS080        DRXTS040        DRXTTFAT        DRXTS100        DRXTS160 
       46.33000        46.10280        45.97521        45.88717        44.57647 
       DRXTSFAT        DRXTP225        DRXTS060        DRXTP226        DRXTP205 
       44.48387        43.54727        43.43777        42.67816        39.74146 
       DRXTALCO        DRXTTHEO        svy_year     demo_gender        DRXTP184 
       38.20010        36.01268        33.39594        27.81569        25.48149 
      demo_race demo_race_black   demo_pregnant 
       22.46557        13.93085         0.00000